In [1]:
stockTicker = "AAPL"
name = 'apple'

In [2]:
import yfinance as yf
import pandas as pd

info = []
info.append(yf.Ticker(stockTicker).info)
df = pd.DataFrame(info)

In [3]:
import datetime

today = datetime.date.today()
startdate = today - datetime.timedelta(days = 5 * 365)

dfh = yf.download(stockTicker, start= startdate, end=today)

[*********************100%***********************]  1 of 1 completed


In [4]:
def cagr(initial, end, time):
    return ((abs(end)/abs(initial))**(1/time))-1

startPrice = dfh.iloc[0,2]
currentPrice = dfh.iloc[len(dfh)-1, 2]

cagrg = cagr(startPrice, currentPrice, 5)

In [5]:
def grahamo():
    eps = df.loc[0, "trailingEps"]
    cv = currentPrice
    v  = (eps * (8.5 + 2*cagrg*100) * 4.4) / 2.57
    return v
grahamo = grahamo()

def grahamr():
    eps = df.loc[0, "trailingEps"]
    cv = currentPrice
    v  = (eps * (7 + cagrg*100) * 4.4) / 2.57
    return v
grahamr = grahamr()

### FCF

In [6]:
pcf = pd.read_html(f'https://www.macrotrends.net/stocks/charts/{stockTicker}/{name}/free-cash-flow')[0].head(10)
pcf.columns = ["Year", "FCF"]
pcf['FCF'] = pcf['FCF'] * 1000000
pcf['growth'] = pcf["FCF"] - pcf["FCF"].shift(-1)
pcf['growthPercentage'] = pcf["growth"] / abs(pcf["FCF"].shift(-1))
pcf

,Year,FCF,growth,growthPercentage
0,2021,9.295300e+10,1.958800e+10,0.266994
1,2020,7.336500e+10,1.446900e+10,0.245670
2,2019,5.889600e+10,-5.225000e+09,-0.081487
3,2018,6.412100e+10,1.234700e+10,0.238479
4,2017,5.177400e+10,-1.723000e+09,-0.032207
5,2016,5.349700e+10,-1.652200e+10,-0.235965
6,2015,7.001900e+10,1.987700e+10,0.396414
7,2014,5.014200e+10,4.641000e+09,0.101998
8,2013,4.550100e+10,2.940000e+09,0.069077
9,2012,4.256100e+10,NaN,NaN


In [7]:
avgGrowth = pcf.iloc[:5,3].mean()
fcf = [pcf.iloc[0,1]*(1+avgGrowth)]
for x in range(9):
    fcf.append(fcf[x]*(1+avgGrowth))

ffcf = pd.DataFrame({'Year':[2021+x for x in range(1,11)],'Free Cash Flow': fcf})
ffcf

,Year,Free Cash Flow
0,2022,1.048036e+11
1,2023,1.181649e+11
2,2024,1.332298e+11
3,2025,1.502152e+11
4,2026,1.693661e+11
5,2027,1.909585e+11
6,2028,2.153038e+11
7,2029,2.427529e+11
8,2030,2.737014e+11
9,2031,3.085955e+11


In [8]:
perpetualGrowthRate = 0.025
discountRate = 0.08

termVal = ffcf.iloc[8,1] * (1+perpetualGrowthRate)/(discountRate-perpetualGrowthRate)
termVal #in billions

5100798074619.757

In [9]:
ffcf["Present Value"] = [ffcf.loc[x-1,'Free Cash Flow']/(1+discountRate)**x for x in range(1, len(ffcf)+1)]
ffcf

,Year,Free Cash Flow,Present Value
0,2022,1.048036e+11,9.704033e+10
1,2023,1.181649e+11,1.013074e+11
2,2024,1.332298e+11,1.057621e+11
3,2025,1.502152e+11,1.104127e+11
4,2026,1.693661e+11,1.152677e+11
5,2027,1.909585e+11,1.203363e+11
6,2028,2.153038e+11,1.256277e+11
7,2029,2.427529e+11,1.311518e+11
8,2030,2.737014e+11,1.369188e+11
9,2031,3.085955e+11,1.429394e+11


In [10]:
bs = yf.Ticker(stockTicker).balance_sheet
bs = bs.reset_index()
bs = bs.set_index('index')
bs.columns = ['2021', '2020', '2019', '2018']

In [14]:
pvTermVal = (termVal/1000000)/(1+discountRate)**10
ccEquiv = bs.loc['Cash', '2021']/1000000
totalDebt = df.loc[0,'totalDebt']/1000000
equityVal = pvTermVal + ccEquiv - totalDebt
sharesOutstanding = df.loc[0,'sharesOutstanding'] /1000000
pricepShare = equityVal / sharesOutstanding 

### Output

In [15]:
valuationModel = ["graham original", 'graham revised', 'DCF']
currentPriceList = [currentPrice for x in range(len(valuationModel))]
intrinsicValue = [grahamo, grahamr, pricepShare]

out = pd.DataFrame({'valuationModel':valuationModel, 'currentPrice':currentPriceList,
      'intrinsicValue':intrinsicValue})

out['projectedReturns'] = out['intrinsicValue'] - out['currentPrice'] 

def gains(arg, comp):
    lst = []
    for x in range(len(out)):
        temp = round((((out.loc[x,arg] / out.loc[x,'currentPrice']) * 100)-100),1) 
        if out.loc[x,comp] < 0:
            lst.append(f"-{str(temp)}%")
        else:
            lst.append(f"{str(temp)}%")
    return lst

out['intrinsicGains'] = gains('intrinsicValue', "projectedReturns")
out["marginalisedValue"] = out.intrinsicValue - (out.intrinsicValue * 0.25)
out["marginalisedReturns"] = out['marginalisedValue'] - out['currentPrice']
out["marginalisedGains"] = gains('marginalisedValue', 'marginalisedReturns')

In [16]:
out

,valuationModel,currentPrice,intrinsicValue,projectedReturns,intrinsicGains,marginalisedValue,marginalisedReturns,marginalisedGains
0,graham original,165.039993,832.086891,667.046897,404.2%,624.065168,459.025175,278.1%
1,graham revised,165.039993,444.363095,279.323102,169.2%,333.272321,168.232328,101.9%
2,DCF,165.039993,139.392285,-25.647708,--15.5%,104.544214,-60.495779,--36.7%
